In [2]:
import numpy as np

class TransformCoordinates:
        
    def __init__(self, fiducial, component, trayPosition = 'P1', isSurvey = False):
        self.tray = fiducial.trayPoints
        self.trayPosition = trayPosition
        self.isSurvey = isSurvey
        if self.trayPosition == 'P1' or self.trayPosition == 'Survey':
            self.position = fiducial.P1 #The two pins
            self.points = np.array(component.pointsP1) #the 4 fiducial points
            self.hgts = np.array(component.heightP1)
            self.hXY = np.array([np.array(component.hXP1), np.array(component.hYP1)])
        elif self.trayPosition == 'P2':
            self.position = fiducial.P2
            self.points = np.array(component.pointsP2)
            self.hgts = np.array(component.heightP2)
            self.hXY = np.array([np.array(component.hXP2), np.array(component.hYP2)])
        self.rad = []
        self.deg = []
        self.center = []
        self.centerPosition = -1.
        self.newPoints = []
        self.hgt = ()
        
    #Calculate the distance between two points or the origin
    def distance(self, tup):
        print(tup)
        if np.array(tup).size == 2:
            return np.sqrt(((tup[0])**2) + ((tup[1])**2))
        else:
            return np.sqrt(((tup[1][0] - tup[0][0])**2) + ((tup[1][1] - tup[0][1])**2))
        
    def vector(self,tup):
        if np.array(tup).size == 2:
          return [tup[0],tup[1]]
        else:
          return [tup[1][0]-tup[0][0],tup[1][1]-tup[0][1]]
    
#     def angleOfFrames(self, Fiducial, baseplate): # Should switch Fiducial and baseplate???
#         f=self.vector(Fiducial)
#         b=self.vector(baseplate)
#         unit_f=f/np.linalg.norm(f)
#         unit_b=b/np.linalg.norm(b)
#         dot_product=np.dot(unit_f,unit_b)
#         angle=np.arccos(dot_product)
#         return np.pi/2 - angle      
    
    def angleOfFrames(self, Fiducial, baseplate): # Should switch Fiducial and baseplate???
        f=self.distance(Fiducial)
        b=self.distance(baseplate)
        unit_f=f/np.linalg.norm(f)
        unit_b=b/np.linalg.norm(b)
        dot_product=np.dot(unit_f,unit_b)
        angle=np.arccos(dot_product)
        return angle
    
    
#   #  Using law of cosines we calculate the angle between the fiducials and the P1/P2 frames
#     def angleOfFrames(self, Fiducial, baseplate):
#         f = self.distance(Fiducial)
#         print("This is Fiducial:", Fiducial)
#         print("This is Baseplate:", baseplate)
#         print("this is f:", f)
#         b = self.distance(baseplate)
#         print("This is b:", b)
#         th1 = 0.
#         #These lines don't do anything - Liam
#         if (baseplate[0][0] - baseplate[1][0]) > 0.:
#             b1, b2 = (baseplate[0][0] - baseplate[1][0]), (baseplate[0][1] - baseplate[1][1])
#         else:
#             b1, b2 = (baseplate[1][0] - baseplate[0][0]), (baseplate[1][1] - baseplate[0][1])
#         #Only b1,b2 used - Liam
#         b1, b2 = (baseplate[0][0] - baseplate[1][0]), (baseplate[0][1] - baseplate[1][1])
#         print("This is Baseplate[0][0]", baseplate[0][0])
#         print("This is Baseplate[0][0]", baseplate[1][0])
#         print("This is Baseplate[0][1]", baseplate[0][1])
#         print("This is Baseplate[1][1]", baseplate[1][1])
#         print("This is b1", b1)
#         print("this is b2:", b2)
#         b3, b4 = (baseplate[1][0] - baseplate[0][0]), (baseplate[1][1] - baseplate[0][1])

#         th1 = Fiducial[1][0]*b1 + Fiducial[1][1]*b2
#         print("This is Fiducial [1][0]:", Fiducial[1][0])
#         print("This is Fiducial [1][1]:", Fiducial[1][1])
#         print("This is th1", th1)
#         #th1 = (a^2+b^2-c^2)/2 in law of cosines apperently - Liam
#         print("Result:", np.pi/2 - np.arccos((th1)/(f*b)))
#         return np.pi/2 - np.arccos((th1)/(f*b))

    #Using a rotation matrix transform the measurements of the points in the fiducial frame
    def transform(self, points, newTheta = 99999.):
        theta = self.angleOfFrames(self.tray, self.position)
        if newTheta != 99999.:
            theta += newTheta
        print("This is theta:", theta)
        c, s = np.cos(theta), np.sin(theta)
        rotate = np.array(((c, -s), (s, c)))
        print("This is self.position:", self.position)
        #print("This is points:", points)
        p = points - np.array(self.position)[0]
        #print("This is p:", p)
        #print(points.shape, rotate.shape)
        #print(p, rotate)
        return p.dot(rotate.T) #.T gives a transpose

    #Calculate the angle offset of the placement of the components
    #also get the center of the square that is made by the fiducial measurements
    def runTransform(self):
        print("This is self.points:", self.points)
        self.newPoints = self.transform(self.points)
        print("This is self.newpoints:", self.newPoints)
        print("This is self.newpoints[0][1]",self.newPoints[0][1])
        print("This is self.newpoints[1][1]",self.newPoints[1][1])
        print("This is self.newpoints[0][0]",self.newPoints[0][0])
        print("This is self.newpoints[1][0]",self.newPoints[1][0])
        for x in range(0, int(len(self.newPoints)/2)):
            r = np.arctan2((self.newPoints[x*2][1] - self.newPoints[x*2 + 1][1]), 
                           (self.newPoints[x*2][0] - self.newPoints[x*2 + 1][0]))
            self.rad.append(r)
            self.deg.append(np.degrees(r))

        self.center = np.sum(self.newPoints, axis=0)/4
        self.centerPosition = self.distance(self.center)
        print("This is self.deg:", self.deg)
        
    #Calculate the mean/min/max of the measured heights of the module or protomodule
    def runHeights(self):
        hMax = np.amax(self.hgts)
        hMin = np.amin(self.hgts)
        hMean = np.mean(self.hgts)
        if not self.isSurvey:
            self.hXY = self.transform(self.hXY.T, -np.pi/2 if self.trayPosition == 'P1' else np.pi/2)
        else:
            self.hXY = self.hXY.T
        #print(self.hXY)
        
        self.hgt = (hMean, abs(hMin - hMean), abs(hMax - hMean))
        
#    def RotatedCenter(self)
        